# [Single-cell RNA-seq reveals cell type–specific molecular and genetic associations to lupus](https://pubmed.ncbi.nlm.nih.gov/35389781/)
GEO: [GSE174188](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE174188)

In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(SingleR)
    library(ShinyCell)
    library(harmony)
    library(glmGamPoi)
    library(anndata)
    })

In [2]:
system("ulimit -m 500000000")

In [4]:
# Read in data
anndata_object <- read_h5ad("data/GSE174188_CLUES1_adjusted.h5ad")

# Create index to split data in half
half_index <- ceiling(nrow(anndata_object) / 2)

# Split data
adata1 <- anndata_object[1:half_index, ]
write_h5ad(adata1, "data/file1.h5ad")

adata2 <- anndata_object[(half_index + 1):nrow(anndata_object), ]
write_h5ad(adata2, "data/file2.h5ad")

In [6]:
# Create Seurat object from first half
part1 <- read_h5ad("data/file1.h5ad")
part1 <- CreateSeuratObject(counts = t(part1$X), meta.data = part1$obs)
part1
save(part1, file = file.path("object", "part1.rda"));rm(part1)

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from PCA to PCA_”
Warning message:
“No columnames present in cell embeddings, setting to 'PCA_1:50'”


ERROR: Error in `colnames<-`(x = `*tmp*`, value = colnames(x = embeddings)): attempt to set 'colnames' on an object with less than two dimensions


In [7]:
# Create Seurat object from second half
part2 <- read_h5ad("data/file2.h5ad")
part2 <- CreateSeuratObject(counts = t(part2$X), meta.data = part2$obs)
part2
save(part2, file = file.path("object", "part2.rda"));rm(part2)

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


An object of class Seurat 
1999 features across 631838 samples within 1 assay 
Active assay: RNA (1999 features, 0 variable features)
 1 layer present: counts

In [8]:
# Merge data
load(file = file.path("object", "part1.rda"));load(file = file.path("object", "part2.rda"))
obj <- merge(x = part1, y = part2);rm(part1, part2)
obj
save(obj, file = file.path("object", "00.rda"))

An object of class Seurat 
1999 features across 1263676 samples within 1 assay 
Active assay: RNA (1999 features, 0 variable features)
 2 layers present: counts.1, counts.2

The data appear to already be normalized

In [9]:
head(obj@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,batch_cov,ind_cov,Processing_Cohort,louvain,cg_cov,ct_cov,L3,ind_cov_batch_cov,Age,Sex,pop_cov,Status,SLE_status
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
CAAGGCCAGTATCGAA-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0,SeuratProject,-91.619885,75,dmx_YS-JY-22_pool6,HC-546,4.0,1,T4,T4_naive,1.0,HC-546:dmx_YS-JY-22_pool6,28.0,Female,Asian,Healthy,Healthy
CTAACTTCAATGAATG-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0,SeuratProject,-10.592551,84,dmx_YS-JY-22_pool6,1132_1132,4.0,7,cM,NA,1.0,1132_1132:dmx_YS-JY-22_pool6,45.0,Female,European,Managed,SLE
AAGTCTGGTCTACCTC-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,SeuratProject,1.238674,143,dmx_AbFlare-3,FLARE006,3.0,0,cM,NA,0.0,FLARE006:dmx_AbFlare-3,34.0,Female,European,Flare,SLE
GGCTCGATCGTTGACA-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0,SeuratProject,-102.855184,60,dmx_YS-JY-20_pool3,1110_1110,4.0,3,B,B_naive,1.0,1110_1110:dmx_YS-JY-20_pool3,71.0,Female,European,Managed,SLE
ACACCGGCACACAGAG-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,SeuratProject,-103.921533,91,dmx_YE110,1479_1479,2.0,2,T4,NA,0.0,1479_1479:dmx_YE110,28.0,Female,Asian,Managed,SLE
TCGTAGATCCTTGGTC-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0-0-0-0,SeuratProject,36.016001,107,dmx_YE_8-2,1334_1334,2.0,4,T8,CytoT_GZMH+,1.0,1334_1334:dmx_YE_8-2,52.0,Female,Asian,Managed,SLE


In [ ]:
# # Clean environment and filterd data
# rm(list = ls())
# load(file = file.path("object", "00.rda"))
# obj <- JoinLayers(obj)
# obj[["percent.mt"]] <- PercentageFeatureSet(object = obj, pattern = "^MT-")
# obj <- subset(obj, subset = nFeature_RNA > 350 & nFeature_RNA < 5000 & percent.mt < 10)
# obj
# save(obj, file = file.path("object", "01.rda"))

In [ ]:
# # Normalize data
# obj <- JoinLayers(obj)
# obj <- SCTransform(obj, ncells = 3000,  variable.features.n = 2000, vst.flavor="v2", method = 'glmGamPoi', conserve.memory = TRUE)
# save(obj, file=file.path("object", "02.rda"))

In [ ]:
# load(file = file.path("object", "00.rda"))
# obj <- JoinLayers(obj)

In [2]:
load(file = file.path("object", "00.rda"))

In [5]:
# RenameGenesSeurat
RenameGenesSeurat <- function(obj = ls.Seurat[[i]], newnames = HGNC.updated[[i]]$Suggested.Symbol) { # Replace gene names in different slots of a Seurat object. Run this before integration. Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data.
  print("Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data.")
  RNA <- obj@assays$RNA

  if (nrow(RNA) == length(newnames)) {
    if (length(RNA@counts)) RNA@counts@Dimnames[[1]]            <- newnames
    if (length(RNA@data)) RNA@data@Dimnames[[1]]                <- newnames
    if (length(RNA@scale.data)) RNA@scale.data@Dimnames[[1]]    <- newnames
  } else {"Unequal gene sets: nrow(RNA) != nrow(newnames)"}
  obj@assays$RNA <- RNA
  return(obj)
}

In [10]:
obj <- JoinLayers(obj)
save(obj, file = file.path("object", "join.rda"))

In [9]:
sc <- RenameGenesSeurat(obj = obj, newnames = rownames(obj))

[1] "Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data."


ERROR: Error in RenameGenesSeurat(obj = obj, newnames = rownames(obj)): no slot of name "counts" for this object of class "Assay5"


In [ ]:
obj <- SetAssayData(object = obj, slot = 'scale.data', new.data = scale.data)

In [ ]:
# Clustering
obj <- ScaleData(obj)

In [5]:
obj <- RunPCA(obj, npcs = 30, verbose = TRUE)

Warning message:
“No layers found matching search pattern provided”


ERROR: [1m[33mError[39m in `PrepDR5()`:[22m
[33m![39m No layer matching pattern 'scale.data' not found. Please run ScaleData and retry


In [ ]:
obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)

In [ ]:
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)

In [ ]:
obj <- FindClusters(obj, resolution = 0.5)
obj

In [ ]:
save(obj, file=file.path("object", "03.rda"))

In [ ]:
DimPlot(obj, group.by = "SLE_status")

In [ ]:
# Clean up metdata columns
obj@meta.data$Orig_CellTypes <- obj@meta.data$ct_cov
obj@meta.data$Race <- obj@meta.data$pop_cov

In [ ]:
# Select needed columns
columns_to_keep <-  c("orig.ident", "Status", "SLE_status", "Sex", "Age", "Race", "Orig_CellTypes", "blueprint.main", "monaco.fine")
obj@meta.data <- obj@meta.data[, columns_to_keep, drop = FALSE]

In [ ]:
seu =  obj
scConf = createConfig(seu)
makeShinyApp(seu, scConf,
             gene.mapping = TRUE,
             shiny.title = "Cellular and genetic correlates of SLE scRNAseq",
             shiny.dir = "cellular_genetic_correlates_SLE_PBMC_scRNAseq_GSE174188/",
             gex.assay = "SCT",
             default.multigene = c('LILRB1', 'PDCD1')) 
system("R -e \"shiny::runApp('cellular_genetic_correlates_SLE_PBMC_scRNAseq_GSE174188')\"")